In [1]:
import os
import sys
import torch
import faiss
import numpy as np
import spacy
import gc
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset, Dataset
# Registers the factory
from scispacy.linking import EntityLinker 

/home/vkarthikeyan1/reddit_health/health_subreddit_small/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


In [2]:
MODEL_NAME = "cambridgeltl/SapBERT-from-PubMedBERT-fulltext"
BATCH_SIZE = 1024       # GPU Batch size
INDEX_CHUNK_SIZE = 50000 # RAM Batch size
TOP_K = 1
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
FAISS_INDEX_STRING = "IVF1024,PQ64" 

print(f"Running on: {DEVICE}")

Running on: cuda


In [3]:

nlp = spacy.load("en_core_sci_lg")
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})
linker = nlp.get_pipe("scispacy_linker")
kb = linker.kb
print(f"   ✅ UMLS KB Loaded. Total concepts: {len(kb.cui_to_entity)}")

print("2. Loading SapBERT Model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(DEVICE)
model.eval()
print("   ✅ SapBERT Loaded.")

   

/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.htm

   ✅ UMLS KB Loaded. Total concepts: 3920422
2. Loading SapBERT Model...
   ✅ SapBERT Loaded.


In [9]:
kb.cui_to_entity['C0000005']

CUI: C0000005, Name: (131)I-Macroaggregated Albumin
Definition: None
TUI(s): T116, T121, T130
Aliases: (total: 1): 
	 (131)I-MAA

In [1]:
import spacy
import scispacy

from scispacy.linking import EntityLinker

nlp = spacy.load("en_core_sci_lg")

# This line takes a while, because we have to download ~1GB of data
# and load a large JSON file (the knowledge base). Be patient!
# Thankfully it should be faster after the first time you use it, because
# the downloads are cached.
# NOTE: The resolve_abbreviations parameter is optional, and requires that
# the AbbreviationDetector pipe has already been added to the pipeline. Adding
# the AbbreviationDetector pipe and setting resolve_abbreviations to True means
# that linking will only be performed on the long form of abbreviations.
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})

doc = nlp("Smoking cures cancer")

# Let's look at a random entity!
entity = doc.ents[1]

print("Name: ", entity)


# Each entity is linked to UMLS with a score
# (currently just char-3gram matching).
linker = nlp.get_pipe("scispacy_linker")
for umls_ent in entity._.kb_ents:
	print(linker.kb.cui_to_entity[umls_ent[0]])



Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib
/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from

Name:  cancer
CUI: C0006826, Name: Malignant Neoplasms
Definition: A tumor composed of atypical neoplastic, often pleomorphic cells that invade other tissues. Malignant neoplasms often metastasize to distant anatomic sites and may recur after excision. The most common malignant neoplasms are carcinomas, Hodgkin and non-Hodgkin lymphomas, leukemias, melanomas, and sarcomas.
TUI(s): T191
Aliases (abbreviated, total: 44): 
	 Neoplasms, Malignant, Cancers, Tumor, malignant, NOS, Malignant neoplasm without specification of site, Neoplastic disease, malignant, CA - Cancer, Malignancy, unspecified site, neoplasm/cancer, malignant neoplasm, Tumor, malignant
CUI: C0998265, Name: Cancer Genus
Definition: None
TUI(s): T204
Aliases: (total: 2): 
	 Cancer, crab
CUI: C1306459, Name: Primary malignant neoplasm
Definition: A malignant tumor at the original site of growth.
TUI(s): T191
Aliases (abbreviated, total: 19): 
	 Malignant neoplasm, primary (morphologic abnormality), malignant neoplasm, Tumor,

: 

In [16]:
from datasets import load_dataset, load_from_disk
ds_triples = load_from_disk("pubmed_output")

In [17]:
df=ds_triples.to_pandas()
df['claim'].value_counts()

claim
na           15941
ambiguous     1098
true           204
false           49
Name: count, dtype: int64

In [18]:
df["triples"][df['claim']=="false"]

298                    [('cipro', 'causes', 'tendonitis')]
508      [('divalproex', 'linked_to', 'violent deaths')...
1031     [('meniscus', 'heals', 'young age'), ('meniscu...
1274     [('metronidazole', 'treats', 'anaerobic gut fl...
1960     [('dairy', 'causes', 'acne'), ('egg whites', '...
2357     [('medication', 'causes', 'sensation in limbs'...
2422     [('MRSA', 'treats', 'clindamycin'), ('MRSA', '...
2682     [('eosinophilic oesophagitis', 'causes', 'diff...
3066     [('celiac disease', 'treats', 'gluten free die...
3325     [('endometriosis', 'treats', 'surgery'), ('end...
3424                  [('toothache', 'treats', 'dentist')]
3438                   [('bronchitis', 'treats', 'strep')]
3487     [('stress', 'causes', 'heart palpitations'), (...
3685     [('celiac disease', 'treats', 'gluten-free die...
4518     [('it', 'cures', 'cancer'), ('it', 'cures', 'v...
4632                   [('antibiotic', 'causes', 'cough')]
4639                   [('antibiotic', 'causes', 'cough'

In [1]:
import pandas as pd

full_df = pd.read_csv("health_misinfo_speed_run.csv")

In [6]:
full_df.count()

author              100000
body                100000
controversiality    100000
created_utc         100000
link_id             100000
score               100000
subreddit           100000
subreddit_id        100000
id                  100000
dtype: int64

In [ ]:
import pandas as pd
from datasets import Dataset
from huggingface_hub import login
login()
df_new= pd.read_parquet("triples_output_100k.parquet")

In [ ]:
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

df_new[:50]

In [4]:

ds=Dataset.from_pandas(df_new)
ds.push_to_hub("Vijayrathank/reddit-health-triples-lg")

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  3.57ba/s]
Processing Files (1 / 1): 100%|██████████| 42.0MB / 42.0MB, 30.0MB/s  
New Data Upload: 100%|██████████| 42.0MB / 42.0MB, 30.0MB/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.63s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/Vijayrathank/reddit-health-triples-lg/commit/ef9a057030af069df9d1d97d9cc5c39a4d821fb9', commit_message='Upload dataset', commit_description='', oid='ef9a057030af069df9d1d97d9cc5c39a4d821fb9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Vijayrathank/reddit-health-triples-lg', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Vijayrathank/reddit-health-triples-lg'), pr_revision=None, pr_num=None)

In [3]:
import pandas as pd
df_cui= pd.read_parquet("../semmed_verifier/triples_cui_mapped.parquet")

In [8]:

def has_triples(val):
    if val is None: return False
    s_val = str(val).strip()
    return s_val != "[]" and s_val != "" and s_val != "None"

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

df_filtered = df_cui[df_cui['triples'].apply(has_triples)].copy()

In [26]:
df_filtered['triples'].size

21777

In [17]:

ds=Dataset.from_pandas(df_cui)
ds.push_to_hub("Vijayrathank/reddit-health-triples-lg")

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  3.37ba/s]
Processing Files (1 / 1): 100%|██████████| 43.3MB / 43.3MB, 1.40MB/s  
New Data Upload: 100%|██████████| 1.37MB / 1.37MB, 1.40MB/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.76s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/Vijayrathank/reddit-health-triples-lg/commit/6ac65b746d489dc068934ed544ecfca031455acb', commit_message='Upload dataset', commit_description='', oid='6ac65b746d489dc068934ed544ecfca031455acb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Vijayrathank/reddit-health-triples-lg', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Vijayrathank/reddit-health-triples-lg'), pr_revision=None, pr_num=None)

In [50]:
import pandas as pd
df_ground= pd.read_parquet("../ground_truth/reddit_health_ground_truth.parquet")


In [51]:
df_ground[['triples','ground_truth_label']]

,triples,ground_truth_label
0,"[(""Ovulation"", ""is_associated_with"", ""Fertilit...",SUPPORTED
1,"[(""Load shedding"", ""prevents"", ""Power outage"")]",UNVERIFIED
2,"[(""Shedding"", ""is_normal"", ""Hair loss"")]",AMBIGUOUS_SUPPORT
3,"[(""Argan oil"", ""improves"", ""Skin hydration""), ...",AMBIGUOUS_SUPPORT
4,"[(""Evergreen tree"", ""is_associated_with"", ""Sno...",SUPPORTED
...,...,...
21772,"[(""Shedding"", ""prevents"", ""Hair matting""), (""S...",AMBIGUOUS_SUPPORT
21773,"[(""Glyphosate"", ""is_more_toxic_than"", ""Caffein...",AMBIGUOUS_SUPPORT
21774,"[(""Dogs"", ""can_lead_to"", ""Diarrhea""), (""Dogs"",...",UNVERIFIED
21775,"[(""Herpes viroids"", ""manifests_as"", ""Sore""), (...",SUPPORTED


In [52]:
import pandas as pd

# --- Configuration ---
FULL_DATASET_PATH = "../dataset/100k_dataset/triples_output_100k.parquet"       # The original data with all 100k rows
GROUND_TRUTH_PATH = "../ground_truth/reddit_health_ground_truth.parquet" # The subset output from your Llama script
OUTPUT_PATH = "../ground_truth/reddit_health_full_annotated.parquet"

def main():
    print("Loading datasets...")
    df_full = pd.read_parquet(FULL_DATASET_PATH)
    df_gt = pd.read_parquet(GROUND_TRUTH_PATH)
    
    print(f"Full Dataset Size: {len(df_full)}")
    print(f"Ground Truth Size: {len(df_gt)}")

    # 1. Select only the necessary columns from Ground Truth to avoid duplicates
    # We only need the ID to match, and the new label columns
    cols_to_merge = ['id', 'ground_truth_label', 'ground_truth_explanation']
    
    # Safety check: Ensure 'id' exists in both
    if 'id' not in df_full.columns or 'id' not in df_gt.columns:
        raise ValueError("Error: 'id' column missing. Cannot merge reliably.")

    # 2. Perform Left Merge
    # how='left' keeps ALL rows from df_full, and attaches labels where ID matches
    print("Merging data...")
    df_merged = df_full.merge(
        df_gt[cols_to_merge], 
        on='id', 
        how='left'
    )

    # 3. Handle rows that were skipped (No Triples)
    # If the extraction model found NO triples, the Llama judge didn't see it.
    # Logically, if there are no triples, the pipeline prediction is "UNVERIFIED".
    # We can fill NaNs with "UNVERIFIED" (or distinct "NO_EXTRACTED_CLAIMS" if you prefer).
    
    print("Filling missing labels for rows without triples...")
    values = {"ground_truth_label": "UNVERIFIED", "ground_truth_explanation": "Skipped: No triples extracted."}
    df_merged = df_merged.fillna(value=values)

    # 4. Save
    print(f"Saving combined dataset to {OUTPUT_PATH}...")
    df_merged.to_parquet(OUTPUT_PATH, index=False)
    print("Done!")
    
    # Optional: Preview
    print("\nLabel Distribution:")
    print(df_merged['ground_truth_label'].value_counts())

if __name__ == "__main__":
    main()

Loading datasets...
Full Dataset Size: 86539
Ground Truth Size: 21777
Merging data...
Filling missing labels for rows without triples...
Saving combined dataset to ../ground_truth/reddit_health_full_annotated.parquet...
Done!

Label Distribution:
ground_truth_label
UNVERIFIED           71398
AMBIGUOUS_SUPPORT     8821
SUPPORTED             6320
Name: count, dtype: int64


In [61]:
df_annotated= pd.read_parquet("../ground_truth/reddit_health_full_annotated.parquet")

df_annotated['ground_truth_label'].value_counts()
df_annotated.columns


Index(['author', 'body', 'controversiality', 'created_utc', 'link_id', 'score',
       'subreddit', 'subreddit_id', 'id', '__index_level_0__', 'triples',
       'ground_truth_label', 'ground_truth_explanation'],
      dtype='object')

In [62]:
df_semmed= pd.read_parquet("../semmed_verifier/triples_semmed_unified_claims.parquet")
df_semmed.columns

Index(['cui_triples', 'claim_verification', 'unified_claim_status'], dtype='object')

In [55]:
import ast
def get_unified_status(verification_list):
    """
    Aggregates micro-claims into a trinary macro-label for the comment.
    Priority: SUPPORTED > AMBIGUOUS_SUPPORT > UNVERIFIED
    """
    verification_list= ast.literal_eval(verification_list)
    # 1. Handle empty/None
    if not isinstance(verification_list, list) or not verification_list:
        return "UNVERIFIED"
    
    # 2. Extract all statuses in this comment
    statuses = set()
    for item in verification_list:
        # Get status, default to unverified if missing
        s = item.get('claim_status', 'UNVERIFIED')
        statuses.add(s)
        
    # 3. Hierarchy Check
    
    # Priority 1: If ANY verified fact exists, the comment is Supported.
    if 'SUPPORTED' in statuses:
        return "SUPPORTED"
        
    # Priority 2: If no strict support, but ambiguous/inverse evidence exists.
    # We group INVERSE_SUPPORT here because it indicates a strong biological link 
    # (just the wrong direction), which fits "Ambiguous" better than "Unverified".
    if 'AMBIGUOUS_SUPPORT' in statuses or 'INVERSE_SUPPORT' in statuses:
        return "AMBIGUOUS_SUPPORT"
        
    # Priority 3: Fallback
    return "UNVERIFIED"
# --- Usage Example ---
# Apply this to your dataframe after the previous step
df_semmed['unified_claim_status'] = df_semmed['claim_verification'].apply(get_unified_status)

In [56]:
df_semmed['unified_claim_status'].value_counts()

unified_claim_status
UNVERIFIED           79106
AMBIGUOUS_SUPPORT     4247
SUPPORTED             3186
Name: count, dtype: int64

In [57]:
df_annotated=pd.read_parquet("../ground_truth/reddit_health_full_annotated.parquet")

df_annotated['ground_truth_label'].value_counts()

ground_truth_label
UNVERIFIED           71398
AMBIGUOUS_SUPPORT     8821
SUPPORTED             6320
Name: count, dtype: int64

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

def evaluate_model_sklearn(y_true, y_pred):
    """
    Computes metrics using scikit-learn.
    y_true: List of ground truth values
    y_pred: List of predicted values
    """
    
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    # Print results
    print("--- Scikit-Learn Evaluation ---")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    
    # Pro Tip: classification_report does all this in one go
    print("\n--- Detailed Report ---")
    print(classification_report(y_true, y_pred))

# Example Usage
if __name__ == "__main__":
    # 1 = Positive Class, 0 = Negative Class
    df_annotated= pd.read_parquet("../ground_truth/reddit_health_full_annotated.parquet")
    df_semmed= pd.read_parquet("../semmed_verifier/triples_semmed_unified_claims.parquet")
    ground_truth = df_annotated['ground_truth_label']
    predictions  = df_semmed['unified_claim_status']
    
    evaluate_model_sklearn(ground_truth, predictions)

--- Scikit-Learn Evaluation ---
Accuracy:  0.8398
Precision: 0.7969
Recall:    0.8398
F1 Score:  0.8103

--- Detailed Report ---
                   precision    recall  f1-score   support

AMBIGUOUS_SUPPORT       0.37      0.18      0.24      8821
        SUPPORTED       0.44      0.22      0.30      6320
       UNVERIFIED       0.88      0.98      0.93     71398

         accuracy                           0.84     86539
        macro avg       0.56      0.46      0.49     86539
     weighted avg       0.80      0.84      0.81     86539



In [74]:
from datasets import load_from_disk
import pandas as pd
pubmed_output= pd.read_parquet("../pubmed_verifier/triples_pubmed_unified_claims.parquet")
pubmed_output['unified_claim_status'].value_counts()


unified_claim_status
UNVERIFIED           82777
SUPPORTED             1893
AMBIGUOUS_SUPPORT     1869
Name: count, dtype: int64

In [ ]:
import pandas as pd

# 1. Load your existing results
input_file = "triples_semmed_unified_claims.parquet"
output_file = "triples_semmed_unified_claims_aligned.parquet"

print(f"Loading {input_file}...")
pubmed_output= load_from_disk("../pubmed_verifier/pubmed_output")

df = pubmed_output.to_pandas()

# 2. Check what current labels look like
print("Original Label Distribution:")
print(df['unified_claim_status'].value_counts())

# 3. Define the Mapping Dictionary
label_map = {
    "true": "SUPPORTED",
    "ambiguous": "AMBIGUOUS_SUPPORT", 
    "false": "UNVERIFIED",  # Science contradicts claim -> Unverified/False Info
    "na": "UNVERIFIED"      # No data -> Unverified
}

# 4. Apply the mapping
# We use .get(x, x) to handle any unexpected labels gracefully (keeps them as is)
df['unified_claim_status'] = df['unified_claim_status'].apply(lambda x: label_map.get(str(x).lower(), "UNVERIFIED"))

# 5. Verify the new labels
print("\nNew Label Distribution:")
print(df['unified_claim_status'].value_counts())

# 6. Save the updated file
df.to_parquet(output_file)
print(f"\nSaved aligned dataset to {output_file}")

In [3]:
import pandas as pd
ground_truth="../ground_truth/reddit_health_full_annotated.parquet"
df_ground=pd.read_parquet(ground_truth)

df_ground['ground_truth_label'].value_counts()




ground_truth_label
UNVERIFIED           71398
AMBIGUOUS_SUPPORT     8821
SUPPORTED             6320
Name: count, dtype: int64

In [5]:
semmed_output= pd.read_parquet("../semmed_verifier/triples_semmed_unified_claims.parquet")
semmed_output['unified_claim_status'].value_counts()

# pubmed_output= pd.read_parquet("./pubmed_verifier/triples_pubmed_unified_claims.parquet")
# pubmed_output['unified_claim_status'].value_counts()



unified_claim_status
UNVERIFIED           79106
AMBIGUOUS_SUPPORT     4247
SUPPORTED             3186
Name: count, dtype: int64